In [1]:
# read excel
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('~/Downloads/sample - arch prospecting.xlsx', sheet_name='sample arch list')

df

,License #,Business Name,First Name,Last Name,Source ref,XXXprojectXXX,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25568.0,"BKSK ARCHITECTS, LLP",TODD,POISSON,https://archpaper.com/2019/11/bksk-and-burohap...,glass shell at Tammany Hall,Example
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14980.0,LALIRE MARCH ARCHITECTS LLP,REX,LALIRE,[to complete[,[to complete[,To complete in test
4,26814.0,"DESIGN AIDD ARCHITECTURE, DPC",ANDREA,HARRIS,[to complete[,[to complete[,To complete in test
5,12394.0,ARCOLOGICA ARCHITECTURE,JORGE,SOSA,[to complete[,[to complete[,To complete in test
6,22510.0,LEESER ARCHITECTURE PLLC,THOMAS,LEESER,[to complete[,[to complete[,To complete in test
7,23295.0,ROSENBAUM DESIGN GROUP,RAND,ROSENBAUM,[to complete[,[to complete[,To complete in test
8,24832.0,"BUILDING STUDIO ARCHITECTS, LLP",JOHN,FIELD,[to complete[,[to complete[,To complete in test
9,15011.0,MICHAEL MUROFF ARCHITECT DPC,MICHAEL,MUROFF,[to complete[,[to complete[,To complete in test


In [106]:
# read architect in row i
i = 3
licenseNo = df['License #'][i]
business = df['Business Name'][i]
firstName = df['First Name'][i]
lastName = df['Last Name'][i]

[google news scraping](https://stackoverflow.com/questions/15550655/web-scraping-google-news-with-python)

In [3]:
query = "%20".join([firstName, lastName, "architect"])
searchURL = f'https://www.google.com/search?pz=1&cf=all&ned=us&hl=en&tbm=nws&gl=us&as_q={query}'

In [6]:
import requests
import bs4

# anti-scraping
userAgent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
response = requests.get(searchURL, headers={'User-Agent': userAgent})
if response.status_code == 200:
    print("Searching ...")
    # print(response.content.decode('utf-8'))
else:
    print(f"Fail to get the url [{response.status_code}]")

Searching ...


In [7]:
# parsing
soup = bs4.BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
# snippet = [a.text for a in soup.find_all('div', class_="Y3v8qd")]
links = [a['href'] for a in soup.find_all('a', href=True)]

In [8]:
import re

# clean links
pattern = r'^https://\w+\.(?!google).\S+'
newsLinks = [re.findall(pattern, a)[0] for a in links if re.findall(pattern, a)]

In [49]:
# filtering news link
# for link i
i = 0
# anti-scraping
newsURL = newsLinks[i]
userAgent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
response = requests.get(newsURL, headers={'User-Agent': userAgent})
if response.status_code == 200:
    html = response.content.decode('utf-8')
else:
    print(f"Fail to get the url [{response.status_code}]")

In [52]:
sources = {}
sources[i] = {}
sources[i]["url"] = newsURL
if re.search(" ".join((firstName, lastName)), html, re.IGNORECASE):
    soup = bs4.BeautifulSoup(html, 'html.parser')
    sources[i]["title"] = soup.find_all('title')[0].text.strip()
    sources[i]["date"] = soup.find_all('date')[0].text.strip()
else:
    sources[i]["error"] = "No match found in page."

In [53]:
sources

{0: {'title': 'BKSK and BuroHappold crown Tammany Hall with a glass shell - Archpaper.com',
  'date': 'November 20, 2019',
  'url': 'https://archpaper.com/2019/11/bksk-and-burohappold-crown-tammany-hall-with-a-glass-shell/'}}

Source 2: architects name on https://newyorkyimby.com/

In [88]:
url = f"https://newyorkyimby.com/?s={firstName}+{lastName}&orderby=post_date&order=desc"

In [89]:
def readURL(url):
    userAgent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    response = requests.get(url, headers={'User-Agent': userAgent})
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print(f"Fail to get the url [{response.status_code}]")
        return False

In [93]:
soup = bs4.BeautifulSoup(readURL(url), 'html.parser')
links = [a['href'] for a in soup.find_all('a', href=True, class_='button-excerpt-more button')]
pattern = r'^https://newyorkyimby.com/[\d]+/[\d]+/\S+'
newsLinks = [re.findall(pattern, a)[0].strip() for a in links if re.findall(pattern, a)]
newsLinks = list(set(newsLinks))
newsLinks = sorted(newsLinks, reverse=True)

In [97]:
newsLinks = sorted(newsLinks, reverse=True)
newsLinks

['https://newyorkyimby.com/2019/10/permits-filed-for-16-east-16th-street-in-union-square-manhattan.html',
 'https://newyorkyimby.com/2016/09/second-try-nets-landmarks-win-for-new-mixed-use-building-at-466-468-columbus-avenue-upper-west-side.html',
 'https://newyorkyimby.com/2016/07/landmarks-wants-refinements-for-bksks-new-mixed-use-building-at-466-468-columbus-avenue-upper-west-side.html',
 'https://newyorkyimby.com/2016/06/landmarks-approves-redevelopment-of-gansevoort-market-block.html',
 'https://newyorkyimby.com/2016/02/landmarks-commissioners-make-feelings-known-as-gansevoort-market-presentation-continues-without-approval.html']

In [101]:
newsURL = newsLinks[0]
news = readURL(newsURL)
source = {}

if re.search(" ".join((firstName, lastName)), news, re.IGNORECASE):
    soup = bs4.BeautifulSoup(news, 'html.parser')
    source["title"] = soup.find_all('title')[0].text.strip()
    source["url"] = newsURL
    
source

{'title': 'Permits Filed for 16 East 16th Street in Union Square, Manhattan - New York YIMBY',
 'url': 'https://newyorkyimby.com/2019/10/permits-filed-for-16-east-16th-street-in-union-square-manhattan.html'}

In [107]:
int(licenseNo)

14980

In [113]:
url = f'http://a810-bisweb.nyc.gov/bisweb/JobsByLicenseNumberServlet?alljappproftitle=RA&all'\
    'japplicnumber={licenseNo}&go20=+GO+&requestid=0&allinquirytype=BXS1LI08'
soup = bs4.BeautifulSoup(readURL(url), 'html.parser')
links = [a['href'] for a in soup.find_all('a', href=True)]

In [114]:
links

['/bisweb/bsqpm01.jsp',
 'http://www1.nyc.gov/site/buildings/index.page',
 'http://www1.nyc.gov/',
 'https://www.nyc.gov/portal/site/nycgov/menuitem.63099911d804683c09416f1076a09da0/',
 'javascript:history.back(1);',
 'http://www1.nyc.gov/site/buildings/index.page',
 'http://www1.nyc.gov/',
 'http://www1.nyc.gov/office-of-the-mayor/index.page',
 'http://www1.nyc.gov/home/privacy-policy.page',
 'http://www1.nyc.gov/home/terms-of-use.page']

In [131]:
"""
Scrape architect info for Arconex.co

"""

import re
import bs4
import requests
import pandas as pd


def readURL(url):
    """
    Return html if url is readable

    """
    userAgent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    response = requests.get(url, headers={'User-Agent': userAgent})
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print(f"Fail to get the url [{response.status_code}]")
        return False


def googleNews(firstName, LastName):
    """
    Google News search for the architect’s name

    """
    print("Searching this person on Google News ...")
    source = {}
    query = "%20".join([firstName, lastName, "architect"])
    searchURL = f'https://www.google.com/search?pz=1&cf=all&ned=us&hl=en&tbm=nws&gl=us&as_q={query}'

    response = readURL(searchURL)
    if response:
        soup = bs4.BeautifulSoup(response, 'html.parser')
    else:
        print("Google search failed.")
    # snippet = [a.text for a in soup.find_all('div', class_="Y3v8qd")]
    links = [a['href'] for a in soup.find_all('a', href=True)]
    # clean links
    pattern = r'^https://\w+\.(?!google).\S+'
    newsLinks = [re.findall(pattern, a)[0] for a in links if re.findall(pattern, a)]

    for i in range(len(newsLinks)):
        newsURL = newsLinks[i]
        news = readURL(newsURL)

        if re.search(" ".join((firstName, lastName)), news, re.IGNORECASE):
            soup = bs4.BeautifulSoup(news, 'html.parser')
            source["title"] = soup.find_all('title')[0].text.strip()
            # source["date"] = soup.find_all('date')[0].text.strip()
            source["url"] = newsURL
            return source

    print("No match found in Google News.")
    return source


def yimby(firstName, LastName):
    """
    Search architects name on https://newyorkyimby.com/

    """
    print("Searching this person on YIMBY ...")
    source = {}
    url = f"https://newyorkyimby.com/?s={firstName}+{lastName}&orderby=post_date&order=desc"
    soup = bs4.BeautifulSoup(readURL(url), 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True, class_='button-excerpt-more button')]
    pattern = r'^https://newyorkyimby.com/[\d]+/[\d]+/\S+'
    newsLinks = [re.findall(pattern, a)[0].strip() for a in links if re.findall(pattern, a)]
    newsLinks = list(set(newsLinks))
    newsLinks = sorted(newsLinks, reverse=True)

    for i in range(len(newsLinks)):
        newsURL = newsLinks[i]
        news = readURL(newsURL)

        if re.search(" ".join((firstName, lastName)), news, re.IGNORECASE):
            soup = bs4.BeautifulSoup(news, 'html.parser')
            source["title"] = soup.find_all('title')[0].text.strip()
            source["url"] = newsURL
            return source

    print("No match found in YIMBY.")
    return source


def dob(licenseNo):
    """
    see if there are any large projects recently approved at the DOB

    """
    url = f'http://a810-bisweb.nyc.gov/bisweb/JobsByLicenseNumberServlet?alljappproftitle=RA&all'\
        'japplicnumber={licenseNo}&go20=+GO+&requestid=0&allinquirytype=BXS1LI08'

# read excel
df = pd.read_excel('~/Downloads/sample - arch prospecting.xlsx', sheet_name='sample arch list')
df = df.dropna().reset_index()

for i in range(len(df)):
    licenseNo = df['License #'][i]
    business = df['Business Name'][i]
    firstName = df['First Name'][i]
    lastName = df['Last Name'][i]
    print(f"=====----- {(i+1)/(len(df)+1)*100} % -----=====")
    print(f"Start collecting {firstName} {lastName}")

    gsource = googleNews(firstName, lastName)
    if gsource:
        df['Source ref'][i] = gsource["url"]
        df['XXXprojectXXX'][i] = gsource["title"]
        print("Got the info from Google News!")
        continue

    ysource = yimby(firstName, lastName)
    if ysource:
        df['Source ref'][i] = ysource["url"]
        df['XXXprojectXXX'][i] = ysource["title"]
        print("Got the info from YIMBY!")
        continue

    # dsource = dob(licenseNo)
    # if ysource:
    #     df['Source ref'] = dsource["url"]
    #     df['XXXprojectXXX'] = dsource["title"]

    continue

print(f"=====----- 100 % -----=====")
print("Seaching complete!")
df

=====----- 10.0 % -----=====
Start collecting TODD POISSON
Searching this person on Google News ...


/Users/solgaleo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/solgaleo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Got the info from Google News!
=====----- 20.0 % -----=====
Start collecting REX LALIRE
Searching this person on Google News ...
No match found in Google News.
Searching this person on YIMBY ...
No match found in YIMBY.
=====----- 30.0 % -----=====
Start collecting ANDREA HARRIS
Searching this person on Google News ...
Got the info from Google News!
=====----- 40.0 % -----=====
Start collecting JORGE SOSA
Searching this person on Google News ...
Got the info from Google News!
=====----- 50.0 % -----=====
Start collecting THOMAS LEESER
Searching this person on Google News ...
Got the info from Google News!
=====----- 60.0 % -----=====
Start collecting RAND ROSENBAUM
Searching this person on Google News ...
Got the info from Google News!
=====----- 70.0 % -----=====
Start collecting JOHN FIELD
Searching this person on Google News ...
No match found in Google News.
Searching this person on YIMBY ...
No match found in YIMBY.
=====----- 80.0 % -----=====
Start collecting MICHAEL MUROFF
Sear

,index,License #,Business Name,First Name,Last Name,Source ref,XXXprojectXXX,Unnamed: 6
0,1,25568.0,"BKSK ARCHITECTS, LLP",TODD,POISSON,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,Example
1,3,14980.0,LALIRE MARCH ARCHITECTS LLP,REX,LALIRE,[to complete[,[to complete[,To complete in test
2,4,26814.0,"DESIGN AIDD ARCHITECTURE, DPC",ANDREA,HARRIS,https://www.brownstoner.com/development/brookl...,Row of Townhouses on State Street in Boerum Hi...,To complete in test
3,5,12394.0,ARCOLOGICA ARCHITECTURE,JORGE,SOSA,https://www.architecturaldigest.com/story/east...,On the Market: A Striking Sun-Filled Beach Hou...,To complete in test
4,6,22510.0,LEESER ARCHITECTURE PLLC,THOMAS,LEESER,https://www.theguardian.com/artanddesign/2020/...,Designer destinations: architects’ favourite h...,To complete in test
5,7,23295.0,ROSENBAUM DESIGN GROUP,RAND,ROSENBAUM,https://www.brooklynpaper.com/not-pretty-in-pink/,Not pretty in pink • Brooklyn Paper,To complete in test
6,8,24832.0,"BUILDING STUDIO ARCHITECTS, LLP",JOHN,FIELD,[to complete[,[to complete[,To complete in test
7,9,15011.0,MICHAEL MUROFF ARCHITECT DPC,MICHAEL,MUROFF,https://archpaper.com/2019/12/bronx-developmen...,See the four must-know developments coming onl...,To complete in test
8,10,19160.0,MCALPINE ARCHITECTURE PC,DAVE,MCALPINE,[to complete[,[to complete[,To complete in test


In [124]:
df.reset_index()

,index,License #,Business Name,First Name,Last Name,Source ref,XXXprojectXXX,Unnamed: 6
0,1,25568.0,"BKSK ARCHITECTS, LLP",TODD,POISSON,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,Example
1,3,14980.0,LALIRE MARCH ARCHITECTS LLP,REX,LALIRE,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
2,4,26814.0,"DESIGN AIDD ARCHITECTURE, DPC",ANDREA,HARRIS,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
3,5,12394.0,ARCOLOGICA ARCHITECTURE,JORGE,SOSA,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
4,6,22510.0,LEESER ARCHITECTURE PLLC,THOMAS,LEESER,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
5,7,23295.0,ROSENBAUM DESIGN GROUP,RAND,ROSENBAUM,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
6,8,24832.0,"BUILDING STUDIO ARCHITECTS, LLP",JOHN,FIELD,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
7,9,15011.0,MICHAEL MUROFF ARCHITECT DPC,MICHAEL,MUROFF,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test
8,10,19160.0,MCALPINE ARCHITECTURE PC,DAVE,MCALPINE,https://archpaper.com/2019/11/bksk-and-burohap...,BKSK and BuroHappold crown Tammany Hall with a...,To complete in test


In [123]:
df['License #'][2]

KeyError: 2